# Test preprocessor module

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import sys
#sys.path.append(os.environ.get('PREPROCPATH'))
sys.path.append('/efs_ecco/mgoldber/project/EH24-PREPROCLLC/preproc/')

from preproc import InSituPreprocessor

%load_ext autoreload
%autoreload 2

In [ ]:
# fastest way for me to get a ds on this machine
import sys
sys.path.append('/efs_ecco/mgoldber/project/EH24-PREPROCLLC/scripts/')
from get_ds import get_ds
grid_ds = get_ds()

Look at dimension attributes

In [ ]:
pkgs = ['profiles', 'obsfit']
for pkg in pkgs:
    ISP = InSituPreprocessor(pkg, grid_ds)
    print(ISP.dims_spatial, ISP.dims_interp, ISP.dims_depth)

Test modules with random observation locations

In [ ]:
def generate_random_points(nobs, lon_range=(-180, 180), lat_range=(-90, 90)):
    lons = np.random.uniform(low=lon_range[0], high=lon_range[1], size=nobs)
    lats = np.random.uniform(low=lat_range[0], high=lat_range[1], size=nobs)
    return lons, lats

nobs = 10000
ungridded_lons, ungridded_lats = generate_random_points(nobs)
ISP = InSituPreprocessor('obsfit')
ISP = InSituPreprocessor('profiles')
ISP.get_obs_point(grid_ds, ungridded_lons, ungridded_lats)
ISP.ds

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.scatter(ISP.ds.sample_lon,ISP.ds.sample_lat, c=ISP.ds.sample_lon, alpha=0.1)
ax1.scatter(ISP.ds.sample_interp_XC11,ISP.ds.sample_interp_YC11, c=ISP.ds.sample_interp_XC11)
ax1.scatter(ISP.ds.sample_interp_XCNINJ,ISP.ds.sample_interp_YCNINJ, c=ISP.ds.sample_interp_XCNINJ)
ax1.set_title('lon fields')
ax2.scatter(ISP.ds.sample_lon,ISP.ds.sample_lat, c=ISP.ds.sample_lat, alpha=0.1)
ax2.scatter(ISP.ds.sample_interp_XC11,ISP.ds.sample_interp_YC11, c=ISP.ds.sample_interp_YC11)
ax2.scatter(ISP.ds.sample_interp_XCNINJ,ISP.ds.sample_interp_YCNINJ, c=ISP.ds.sample_interp_YCNINJ)
ax2.set_title('lat fields')


From here, one can save the object to netcdf
```
insitu_obj = ISP.ds.copy()
dir_out = '/path/to/insitu/dir/'
fout_out = 'insitu_obj.nc'
insitu_obj.to_netcdf(dir_out + fout)
```

In [ ]:
# from ecco_v4_py.tile_plot import *
# _, XC_wm = plot_tiles(grid_ds.XC, layout='latlon', rotate_to_latlon=True, show_tile_labels=False,
#                       show_colorbar=True)

# Example: profiles in labsea

In [ ]:
ungridded_lons, ungridded_lats = ([310], [55]) # see labsea input/data for XGorigin
ISP = InSituPreprocessor('profiles')
ISP.get_obs_point(grid_ds, ungridded_lons, ungridded_lats)
ISP.ds

In [ ]:
prof_depth = ds.Z.values
ISP.ds['prof_depth'] = xr.DataArray(prof_depth, dims=['iDEPTH'])
prof_T = np.zeros_like(prof_depth);
prof_T[0] = 20
prof_T = prof_T[None, :]
prof_Tweight = np.zeros_like(prof_depth);
prof_Tweight[0] = 1.
prof_Tweight = prof_Tweight[None, :]

ISP.ds['prof_T'] = xr.DataArray(prof_T, dims=['iPROF', 'iDEPTH'])
ISP.ds['prof_Tweight'] = xr.DataArray(prof_Tweight, dims=['iPROF', 'iDEPTH'])

# temporal info
ISP.ds['prof_YYYYMMDD'] = xr.DataArray(np.array([19790101]), dims=['iPROF'])
ISP.ds['prof_HHMMSS'] = xr.DataArray(np.array([10000]), dims=['iPROF'])

prof_interp_weights = np.ones((1, 4)) * 0.25
ISP.ds['prof_interp_weights'] = xr.DataArray(prof_interp_weights, dims=['iPROF', 'iINTERP'])


In [ ]:
dir_out = '/home/jovyan/'
#/efs_ecco/mgoldber/MITgcm_c68i/verification/lab_sea/ecco_hackathon/input_prof/'
ISP.ds.to_netcdf(dir_out + 'prof_dummy_2x2.nc')